<h1 align="center"><a href="https://github.com/sborquez/her2bdl"> Her2BDL</a> - Her2 Bayesian Deep Learning</h1>


<br>
<center>
<img src="images/utfsm.png" width="50%", align="center"/>
</center>
<h2 align="center">Model Evaluation</h2>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez.g@gmail.com">sebastian.borquez.g@gmail.com</a> - utfsm - October 2020.</i>
</center>


# Setup Notebook

The first step is to sync this notebook with Google Drive, and change directory to gerumo repository.

## (Option A) Colab Setup

Connecto to your `Google Drive` and install some dependencies.

In [ ]:
!pip list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/<Path to Project>"
!ls

## (Option B) Local Setup

Change working directory to project root.

In [ ]:
%cd ..

## Import Modules

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_DISABLE_PTX_JIT'] = "1"

# her2bdl packege
from her2bdl import *

# Adhoc modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
from IPython.core.display import display, HTML

In [ ]:
import os
os.environ["WANDB_MODE"] = 'dryrun'

# Model Evaluation

Load and predict with your trained models. Evaluate and plot some metrics.

In [ ]:
experiments = Path("/home/asuka/projects/her2bdl/train/experiments/config")

# Binary
config_file = experiments / "binary_classification_evaluation/efficientnet_b0_binary_classifier.yaml"

# HER2
#config_file = experiments / "baseline_evaluation/hdxconv_c876_d02_mixture.yaml"

In [ ]:
config = load_config_file(config_file)
run_dir = setup_experiment(config, mode="evaluation")

# Experiment paths and indentifiers
experiments_folder = config["experiment"]["experiments_folder"]
experiment_name    = config["experiment"]["name"]
experiment_folder  = Path(experiments_folder) / experiment_name
run_id             = config["experiment"]["run_id"]

# Evaluation parameters
batch_size  = config["evaluation"]["batch_size"]
evaluate_classification = config["evaluation"]["evaluate_classification"]
evaluate_aleatoric_uncertainty = config["evaluation"]["evaluate_aleatoric_uncertainty"]
evaluate_aggregation = config["evaluation"]["evaluate_aggregation"]
# Dataset
data_configuration = config["data"]
test_, input_shape, num_classes, labels = setup_generators(
    batch_size=batch_size, test_dataset=True,  **data_configuration
)
(test_dataset, steps_per_epoch) = test_
# Model architecture
model_configuration = config["model"]
model = setup_model(input_shape, num_classes, **model_configuration)

In [ ]:
# Evaluation Logguer
logger_configuration = config["evaluation"]["experiment_logger"]
logger = setup_evaluation_logger(
    model_name=experiment_name,
    labels=labels,
    run_dir=run_dir,
    **logger_configuration
)

## Evaluate Classification

In [ ]:
logger_configuration['classification_metrics']

In [ ]:
results = model.predict_with_epistemic_uncertainty(test_dataset, include_data=True)
data, predictions_results, uncertainty_results = results

In [ ]:
logger.log_classification_metrics(data, predictions_results, uncertainty_results)

## Evaluate Aleatoric Model

In [ ]:
logger_configuration['aleatoric_uncertainty']

In [ ]:
aleatoric_model = model.get_aleatoric_model()
results_aleatoric = aleatoric_model.predict_with_aleatoric_uncertainty(test_dataset, include_data=True)
aleatoric_data, aleatoric_predictions_results, aleatoric_uncertainty_results = results_aleatoric

In [ ]:
logger.log_aleatoric_uncertainty(aleatoric_data, aleatoric_predictions_results, aleatoric_uncertainty_results)

## Evaluate Aggregation 

In [ ]:
import wandb
wandb.finish()
os.environ["WANDB_MODE"] = ""